In [93]:
import pandas as pd
import numpy as np

In [94]:
# DROP COLUMNS
notas_red_comp = ['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']
presenca_na_prova = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT']
codigo_dos_cadernos = ['CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT']

notas_nas_provas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT']

columns_to_use = ['NU_INSCRICAO', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']


## Visualizando nosso dataset de treino

In [95]:

df = pd.read_csv("train.csv").loc[:14000,columns_to_use].dropna()
df_sem_competencia_red = df.drop(columns=notas_red_comp)
columns_name = list(df.columns)
df_sem_competencia_red

,NU_INSCRICAO,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_REDACAO
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,ABACCABDBEDEDEEDDDBBBDAABABECDCCCAAABCBEEABCA,BBEBCAEBCEDBDDEBCACDBCBDABDDCDBEDCEAEABCAADAC,99999DCABCABCBBCADBCBBABAEADBEEBEDAAABAEAECCBB...,AECCCBABDCADDBCBCCADADBBAADEBCEEBABCDDEAABCAA,1,BCADBCEDCCBADBDCADBEBDBEADCCAEACDBEDBCACAEBEE,BCBCCADCCBEDDBEEADBDCECCBBECAEEDDADCDDACAADBA,CEDEACCBEBDBABECABDADEADCEDCBEBBCEBAAEBAAECBBC...,DADCCECBDECADCBACDBCEEABDDAABDBCEADDBECEBAEBC,1.0,520.0
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,EDAAABDABADBCCAEDCDDECABADBEECBBAEEDCABCAABBC,ADCAABAADCEDAACDACEEDACBDCBACEDEDBACAADADABDC,99999ACBACDBDBCEADDAEEADCCEACEEDBECAAAACBECDAC...,AEDACBDECEDAEEAEDDEBDECDCECACADCACACAABDACEAB,1,EBEECADBCABECDDBEADCCABDEABEBCADBADDCCCEDBBCA,DCECBAAADCBBECAEDCEDDACCBDDACEDDBADBCADBCBCEE,CEDEACCBEBDBABECABDADEADCEDCBEBBCEBAAEBAAECBBC...,DADCCECBDECADCBACDBCEEABDDAABDBCEADDBECEBAEBC,1.0,580.0
5,a37c99ec251d4f6e8ddbeabadf1c87fdbfddc4d1,AEABBBABBDBCDDCBCABDEBCDCBEDCACACDEBAECCDEDCD,EBECEBBDECCDACCCACBDBECBBAECDAAACAABCBEAAACEE,ABBDD99999EBEEBCAECAEADEACBCDEEBCABBACDEABAECA...,DCCEDACCCDCEBDABDBCCCDEBECBDACEDCCDBABAADCBCC,0,DBDCCACADBADBCABDEBEECECDDBCADCCBEBBEDBEAEACA,EEBCBCEDDBDDACCCBCADADBAADEDDADCECAECBBBADCEC,EDECACCBBEDEBBBDABCBBCCEBADCEBAAEAAEDCBCABDBAB...,CEECDBDADEBCCEBABECEACBDABDBDDABDDECADCCEACBA,1.0,620.0
6,63b4e128e9ffe8ab27f5d093db1976ef4f353e0a,AADABBDBDADEDABCDEDBDCCBBBBBACCACAABADCCBCCAA,EBDCEABCACEEDBAABECEAECEDAEAEEBCADDECAEBEDAAD,EDDAD99999CAAEDCEBCCDEBDDCBDDAABAAAEDDCADDEABA...,EACBBDCBAAABBDDADDBBBBCBBCBAAAAADDBAACBBAADAA,0,EADBDCCBADBEBCEBCABDEBEEBECADDBEACACDDBCADCCA,EDDBDDACDCEDDACBBBCBCDCECADBEEBAAADECAECCBCAD,EECDABBECCDAAECBBCBDABBAAEEBBDCBADEBABEADCECEB...,CCEEBCCEBAABDDECADCEECDBCBDBECBADDACEAABDBDAD,1.0,560.0
7,2eb189d4912f64b19d1967e8e84b6141aba18770,EBEEBDAACCBAAADBECCCBABDDADCCAABEEADBACDBBBDA,DCEEBAAAEABBECAEECECDADBCDDACBDEAADBABDEADEEA,DECEA99999DADDBEBDAABEDCEBBDBBBDEBABEBECEBBADA...,DDBAECBCEEABDBAECBDECBECABECBBDDBCBDBBCCCDCAD,0,EBEECADBCABECDDBEADCCABDEABEBCADBADDCCCEDBBCA,DCECBAAADCBBECAEDCEDDACCBDDACEDDBADBCADBCBCEE,DECEACCEBBDADEBABEAAEDCBCABDCEBADCEBAAECBBCBDA...,CBAACCEABDDCDBDDCEAECADBECEBACBDABDBEBCDADCEE,1.0,620.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13725,aadf671ef8c1c6efa42e69432093ab83c17a52ac,DDABDCDBBDBACDCECDCBEBCDCEBBDABDCACDEABDECDBC,BDEDCADCEBBCAAEBDBECACACABDBDBABCABECADDDBEAC,ACDDB99999BBDACBECAAEDBCCDAECBDECCABDDAEEEBAEC...,ECDABADBAEDDBEAEADCACDBCCADCDBCCEBEDBDBDABDCA,0,EADBDCCBADBEBCEBCABDEBEEBECADDBEACACDDBCADCCA,EDDBDDACDCEDDACBBBCBCDCECADBEEBAAADECAECCBCAD,CEDEACCBEBDBABECABDADEADCEDCBEBBCEBAAEBAAECBBC...,DADCCECBDECADCBACDBCEEABDDAABDBCEADDBECEBAEBC,1.0,440.0
13726,461f7ef2753d9d5fa0b054930d6b0e0eec318e81,BDBBBDBCEAECCCAEAADEEBEADADCCACCABDDCAEDCDBDC,DCECBBBCAAEBCCAABBEDCDCBDBAABADDDADBAADADEBEB,99999CCABCABDEEEECCCECADDBBECCEAAACAABABABBDDB...,BAAEDBCB.CABCABEEBCAECAABCBDACEACCDCBBEAAACEA,1,EBEECADBCABECDDBEADCCABDEABEBCADBADDCCCEDBBCA,DCECBAAADCBBECAEDCEDDACCBDDACEDDBADBCADBCBCEE,EDECACCBBEDEBBBDABCBBCCEBADCEBAAEAAEDCBCABDBAB...,CEECDBDADEBCCEBABECEACBDABDBDDABDDECADCCEACBA,1.0,560.0
13727,5354503700ecf2388f68990435390b6e1ad454ee,AEDACDAABBDEEDBDDAEAADCAEBAAAAEABCDDAAEECECBB,ABBECBBAEEDBCCCDAAADAEBAADCAAADDBBABEBBAADEEA,DBECA99999AEBBEAACDBBBEECADADBEBDAACCDBDEBAECE...,DBACDBEAEAEDDEDDDBCECBDAEDBABDDBBCDEAABDCDDBD,0,DBDCCACADBADBCABDEBEECECDDBCADCCBEBBEDBEAEACA,EEBCBCEDDBDDACCCBCADADBAADEDDADCECAECBBBADCEC,EDECACCBBEDEBBBDABCBBCCEBADCEBAAEAAEDCBCABDBAB...,CEECDBDADEBCCEBABECEACBDABDBDDABDDECADCCEACBA,1.0,600.0
13728,c7a9e94a42bd76b7978f13c013a61702e36cc42c,AECECBACCBCCBBADCDBBAECABDEABDDDAEDCCDBDBABCB,BDDDBDBCCBECDACEBEDEACCEABDBEEBAAAEACADCACADD,99999BCBCABAACCCBCCEEABEACEECAACAAEBABEDBDAEEB...,DBABECBCBEBBDABCAEBCCADEBABCDBCDBBCAEECBBCABB,1,EADBDCCBADBEBCEBCABDEBEEBECADDBEACACDDBCADCCA,EDDBDDACDCEDDACBBBCBCDC